In [ ]:
#%pip install google-cloud-translate==2.0.1

In [1]:
from google.cloud import translate

In [14]:
import os
import time

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(os.getcwd(), "service.json")

In [15]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'f:\\my_year2023\\PLN\\code\\MARIAN\\scripts\\benchmark\\service.json'

In [16]:
DATA_DIR = os.path.join('..', '..', 'artifacts', 'data')
TEST_DIR = os.path.join(DATA_DIR, 'test')
TEST_GN_FILEPATH = os.path.join(TEST_DIR, 'test.gn')
TEST_ES_FILEPATH = os.path.join(TEST_DIR, 'test.es')

In [62]:
def translate_text(text: list[str], src: str, trg: str, parent: str):
    client = translate.TranslationServiceClient()

    text = [t.encode("utf-8") for t in text]

    response = client.translate_text(parent=parent,
                                     contents=text,
                                     source_language_code=src,
                                     target_language_code=trg,
                                     mime_type="text/plain") # mime types: text/plain, text/html

    translation = response
    return translation

In [63]:
example = translate_text(["Ahora es el turno de Cartes!", "chau"], 
                         'es', 
                         'gn')
example

translations {
  translated_text: "\302\241Ko\'\303\241g\303\243 ha\'e Cartes turno!"
}
translations {
  translated_text: "jajohecha peve"
}

In [64]:
def process_output(translations):
    translations = translations.translations
    return [translation.translated_text for translation in translations]
    
process_output(example)

["¡Ko'ágã ha'e Cartes turno!", 'jajohecha peve']

In [65]:
gn_test_text = []
es_test_text = []
data = {
    "gn": {
        "text": [],
        "translation_text": [],
        "path": TEST_GN_FILEPATH
    },
    "es": {
        "text": [],
        "translation_text": [],
        "path": TEST_ES_FILEPATH
    }
}

for language in data:
    with open(data[language]["path"], 'r', encoding='utf-8') as f:
        data[language]["text"] = f.readlines()

In [69]:
data["gn"]["text"][:5]

["Ko'ágã Cartes turno\n",
 "Movilización ciudadana oñembyaty ko'ã tendáre residencia mburuvicha Horacio Cartes oîva avenida España.\n",
 "Ko'ã manifestante omoî iñakãre ome'ê Yasyretã Argentina-pe, nota rupive.\n",
 "Tavaygua ko'ã ary oñembyatýva calle España ha Soldado Desconocido  100 metro rupi oikohápe Cartes, ombotývo tape sapy'a py'a tavaguasúpe.\n",
 'Ojehecha despliegue policial ohapejokóva manifestantekuéra Cartes róga rovái, ha ijerére.\n']

In [74]:
batch_size = 100

for idx, language in enumerate(data):
    for idy, batch_text in enumerate([data[language]["text"][i:i+batch_size] \
                                      for i in range(0, 
                                                     len(data[language]["text"]), 
                                                     batch_size)]):
        translated_text = translate_text(batch_text, 
                                         language, list(data.keys())[idx-1], 
                                         parent=os.environ["PARENT"])
        processed_translation = process_output(translated_text)
        data[language]["translation_text"].extend(processed_translation)
        print('Batch completed: ', idy, '-', batch_size*(idy+1))
        time.sleep(1)
                                       
    with open(f'google_{language}_{list(data.keys())[idx-1]}.txt', 'w', encoding='utf-8') as f:
        f.writelines(''.join(data[language]["translation_text"]))

Batch completed:  0 - 100
Batch completed:  1 - 200
Batch completed:  2 - 300
Batch completed:  3 - 400
Batch completed:  4 - 500
Batch completed:  5 - 600
Batch completed:  6 - 700
Batch completed:  7 - 800
Batch completed:  8 - 900
Batch completed:  9 - 1000
Batch completed:  10 - 1100
Batch completed:  11 - 1200
Batch completed:  12 - 1300
Batch completed:  13 - 1400
Batch completed:  14 - 1500
Batch completed:  15 - 1600
Batch completed:  16 - 1700
Batch completed:  17 - 1800
Batch completed:  18 - 1900
Batch completed:  19 - 2000
Batch completed:  20 - 2100
Batch completed:  21 - 2200
Batch completed:  22 - 2300
Batch completed:  23 - 2400
Batch completed:  24 - 2500
Batch completed:  25 - 2600
Batch completed:  26 - 2700
Batch completed:  27 - 2800
Batch completed:  28 - 2900
Batch completed:  29 - 3000
Batch completed:  30 - 3100
Batch completed:  31 - 3200
Batch completed:  32 - 3300
Batch completed:  33 - 3400
Batch completed:  34 - 3500
Batch completed:  35 - 3600
Batch compl